In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from collections import Counter
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

## Additional RAM

In [ ]:
# But it is no more proposed get more RAM in a free way
[1]*10**10

## data

In [ ]:
books = "/content/drive/MyDrive/ProjetML/Base_BookCrossing/Base_completee/bothWebSites_InternetSearch_AllBooks_cleaned_w2v.csv"
ratings = "/content/drive/MyDrive/ProjetML/BX-Book-Ratings.csv"

In [ ]:
#Cleaned data base with scrapped books
df_books = pd.read_csv(books, sep = "|")

In [ ]:
#ratings of Book Crossing
df_ratings = pd.read_csv(ratings, sep = ";", error_bad_lines=False, encoding="latin-1")

In [ ]:
df_ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
df_books.head(1)

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards
0,0439023483,9.780439e+12,NaN,The Hunger Games : The First Book of the Hunge...,Suzanne Collins,2008-01-01,NaN,juvenile fict,NaN,NaN,en,http://books.google.com/books/content?id=sJdUA...,374.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_ratings[df_ratings.ISBN == "000648302X"]

,User-ID,ISBN,Book-Rating
45384,11676,000648302X,8
174443,37950,000648302X,9
373708,89624,000648302X,9
412327,98544,000648302X,0
417691,99328,000648302X,3
496961,119654,000648302X,8
499864,120610,000648302X,7
654880,158589,000648302X,8
659263,160016,000648302X,4
718973,174092,000648302X,0


## Tests merge

In [ ]:
DF_CatUsers = pd.DataFrame( {"UserID": [1,1,1,2,2,3,3,3,3], 
                             "ISBN": ["ISBN1", "ISBN2", "ISBN3", "ISBN4", "ISBN5", "ISBN2", "ISBN3", "ISBN5", "ISBN6"], 
                             "Ratings": [4,5,3,2,1,4,5,4,4]})
DF_CatUsers

In [ ]:
DF_CatBooks = pd.DataFrame( {"ISBN": ["ISBN1", "ISBN2", "ISBN3", "ISBN4", "ISBN5", "ISBN6", "ISBN7", "ISBN8"], 
                             "features": ["Blabla1", "Blabla2", "Blabla3", "Blabla4", "Blabla5", "Blabla6", "Blabla7", "Blabla8"]})
DF_CatBooks

In [ ]:
pd.merge(DF_CatUsers, DF_CatBooks, on = "ISBN")

## Merge

In [ ]:
#Join of ratings and books data base, according to ISBN which is a common column
df_bert = pd.merge(df_ratings, df_books, left_on = "ISBN", right_on = "isbn")  
len(df_bert)

110801

In [ ]:
#I will only embeds lines with a not empty book_description
lineOK = df_bert[df_bert["book_description"].isna() == False]
len(lineOK)

75976

In [ ]:
#Creation of an additional column in order to put the future book_description embedding
df_bert = df_bert.assign(description_bert = 0)

In [ ]:
lineOK.index[997:1002]

Int64Index([1549, 1550, 1551, 1552, 1553], dtype='int64')

In [ ]:
len(df_bert)

110801

In [ ]:
df_bert.head()

,User-ID,ISBN,Book-Rating,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards,description_bert
0,276725,034545104X,0,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,0
1,2313,034545104X,5,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,0
2,6543,034545104X,0,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,0
3,8680,034545104X,5,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,0
4,10314,034545104X,9,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,0


In [ ]:
len(df_bert)

110801

## Test descriptions

In [ ]:
for d in df_books["book_description"][:3]:
    print(d)

## Bert

### Loading

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 8.7MB/s 
     |████████████████████████████████| 901kB 53.1MB/s 
     |████████████████████████████████| 3.3MB 54.2MB/s 


In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
#I will use an already trained BERT network
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

### Tokenizer and embedding

In [ ]:
#Use of pretrained model
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

In [ ]:
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
#With use of GPU I will save lots of time
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
#execution in 

with torch.no_grad():
  for i_line in lineOK.index[0:2]:
    
    #one book description, potentially composed of several sentences
    sent_list = df_bert.book_description[i_line].split(".")
    #I only keep the not empty results of the previous split
    sent_list = [sent for sent in sent_list if len(sent) > 3]

    #Output of sentences tokenization
    input_ids = []
    attention_masks = []

    #For each sentence in the current book_description
    for sent in sent_list:

      #print(len(sent.split()))
      #Tokenize all of the sentences and map the tokens to thier word IDs.
      encoded_dict = tokenizer.encode_plus(
                                  sent,                      # Sentence to encode.
                                  add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                                  max_length = 64,           # Pad & truncate all sentences.
                                  pad_to_max_length = True,
                                  return_attention_mask = True,   # Construct attn. masks.
                                  return_tensors = 'pt',     # Return pytorch tensors.
                        )
              
      # Add the encoded sentence to the list.    
      input_ids.append(encoded_dict['input_ids'])
              
      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    input_ids_t = torch.cat(input_ids, dim = 0)
    attention_masks_t = torch.cat(attention_masks, dim = 0)

    #use Bert to embedds each tokenized sentences of the current book_description
    outputs = model(input_ids_t.to(device), attention_masks_t.to(device))
    hidden_states = outputs[2]

    # `hidden_states`       has shape [13 layers, len(input_ids), 64, 768] 
    # => hidden_states[-2]  has shape [len(input_ids), 64, 768] 
    token_vecs = hidden_states[-2]
    print(token_vecs.shape)

    #mean of each embedded sentences of the current book_description
    #=> sentence_embedding  has shape [64, 768]: 
    sentence_embedding = torch.mean(token_vecs, dim = 0)
    print(sentence_embedding.shape)

    #I want a flat tensor with the same information: 64*768 values = 49 152 values !
    sentence_embedding = sentence_embedding.view(-1)
    print(sentence_embedding.size())

    #I get the values from GPU, and I put it in df_bert.description_bert column
    sentence_embedding = sentence_embedding.cpu().numpy()
    l = len(sentence_embedding)
    df_bert.description_bert[i_line] = [str(sentence_embedding[i]) for i in range(l)]


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats i

In [ ]:
# Print sentence 0, now as a list of IDs.
print('Original: ', df_bert.book_description[0])
print('Token IDs:', df_bert.description_bert[0])

Original:  Twenty years after having an affair with legendary artist George Gabriel as a college student, Genny Haviland is accused of drugging and suffocating the painter, despite her claim that she only did it to ease her former lover to a painless ...
Token IDs: tensor([  101,  3174,  2086,  2044,  2383,  2019,  6771,  2007,  8987,  3063,
         2577,  6127,  2004,  1037,  2267,  3076,  1010,  8991,  4890,  5292,
        14762,  5685,  2003,  5496,  1997,  4319,  4726,  1998, 10514,  4246,
        27483,  1996,  5276,  1010,  2750,  2014,  4366,  2008,  2016,  2069,
         2106,  2009,  2000,  7496,  2014,  2280,  7089,  2000,  1037,  3255,
         3238,  1012,  1012,  1012,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0])


In [ ]:
df_bert.to_csv("/content/drive/MyDrive/ProjetML/Books_description_bert.csv", sep = "|", index = False)

# Train / Test data creation

In [ ]:
#We get the bert embedding of book_description 
df_bert = pd.read_csv("/content/drive/MyDrive/ProjetML/Books_description_bert.csv", sep = "|")

In [ ]:
#We only use the not empty book_description lines
lineOK = df_bert[df_bert["book_description"].isna() == False]
len(lineOK)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


29932

In [ ]:
print(len(df_bert))
df_bert.head()

110801


,User-ID,ISBN,Book-Rating,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards,description_bert
0,276725,034545104X,0,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,"['-0.18947688', '0.025109343', '0.19813403', '..."
1,2313,034545104X,5,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,"['-0.18947688', '0.025109343', '0.19813403', '..."
2,6543,034545104X,0,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,"['-0.18947688', '0.025109343', '0.19813403', '..."
3,8680,034545104X,5,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,"['-0.18947688', '0.025109343', '0.19813403', '..."
4,10314,034545104X,9,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,"['-0.18947688', '0.025109343', '0.19813403', '..."


In [ ]:
X_data = df_bert[["User-ID", "description_bert"]]
X_data.head()

,User-ID,description_bert
0,276725,"['-0.18947688', '0.025109343', '0.19813403', '..."
1,2313,"['-0.18947688', '0.025109343', '0.19813403', '..."
2,6543,"['-0.18947688', '0.025109343', '0.19813403', '..."
3,8680,"['-0.18947688', '0.025109343', '0.19813403', '..."
4,10314,"['-0.18947688', '0.025109343', '0.19813403', '..."


In [ ]:
Y_data = df_bert["Book-Rating"]
Y_data.head()

0    0
1    5
2    0
3    5
4    9
Name: Book-Rating, dtype: int64

In [ ]:
len(X_data)

110801

In [ ]:
MIN = min(df_bert["User-ID"])
MAX = max(df_bert["User-ID"])
print("min:", MIN)
print("max:", MAX)

min: 8
max: 278854


In [ ]:
#Transformation of a X_data line into a vector 
#beacause bert embedding has been saved as a list of strings
def vectBert(l):
  #bert embedding of dim768 and User-ID
  b = np.zeros(768+1)

  #1st copy of User-ID
  b[0] = (l[0]-MIN)/(MAX-MIN)

  #THen handling of list of strings
  v = X_data.iloc[0][1].split("'")
  b[1:] = [float(v[i]) for i in range(1, len(v), 2)]
  
  return b

In [ ]:
print(len(vectBert(X_data.iloc[0])))
vectBert(X_data.iloc[0])[0:5]

769


array([ 0.99236496, -0.18947688,  0.02510934,  0.19813403, -0.4776927 ])

In [ ]:
#treatment of all lines of X_data such that X_data.book_description is not empty
def matBert(l, df):
  m = np.zeros((len(l), (768+1)))
  
  ind = 0
  for i in l.index:
      m[ind] = vectBert(df.iloc[i])
      ind = ind + 1
  
  return m

In [ ]:
#It takes about 1min for about 75 000 lines
mat_X_data = matBert(lineOK, X_data)
mat_X_data.shape

(29932, 769)

In [ ]:
#treatment of all corresponding lines of Y_data
#=> same lines as lines for which X_data.book_description is not empty
def vectLabels(l, df):
  m = np.zeros((len(l), 1), dtype = int)
  
  ind = 0
  for i in l.index:
      m[ind] = df.iloc[i]
      ind = ind + 1
  
  return m

In [ ]:
vect_Y_data = vectLabels(lineOK, Y_data)
vect_Y_data.shape

(29932, 1)

In [ ]:
(X_train, X_test, y_train, y_test) = train_test_split(mat_X_data, vect_Y_data,          
                                           test_size=.2, stratify = vect_Y_data)

In [ ]:
print("len(lineOK)", len(lineOK), "\n")

print("X_train.shape", X_train.shape)
print("X_test.shape", X_test.shape, "\n")

print("y_train.shape", y_train.shape)
print("y_test.shape", y_test.shape)

len(lineOK) 29932 

X_train.shape (23945, 769)
X_test.shape (5987, 769) 

y_train.shape (23945, 1)
y_test.shape (5987, 1)


In [ ]:
type(X_train)

numpy.ndarray

In [ ]:
X_train[:5]

array([[ 0.19897004, -0.18947688,  0.02510934, ..., -0.07602315,
         0.05378568, -0.02498486],
       [ 0.24817641, -0.18947688,  0.02510934, ..., -0.07602315,
         0.05378568, -0.02498486],
       [ 0.55151589, -0.18947688,  0.02510934, ..., -0.07602315,
         0.05378568, -0.02498486],
       [ 0.93762148, -0.18947688,  0.02510934, ..., -0.07602315,
         0.05378568, -0.02498486],
       [ 0.49270565, -0.18947688,  0.02510934, ..., -0.07602315,
         0.05378568, -0.02498486]])

In [ ]:
y_train[0]

array([0])

# LSTM

In [ ]:
embedding_dim = 64*768 = 49 152
HIDDEN_DIM = 120

class LSTMratings(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, tagset_size):
        super(LSTMratings, self).__init__()
        self.hidden_dim = hidden_dim

        # Complete the following line
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        #Each sentence is of dim [64, 678]
        sentence_mat = self.transform_sent(sentence)
        
        #inputs  are of size [64, batch_size, embedding of size 678]
        lstm_out, _ = self.lstm(embeds.view(64, -1, 678))          
        #outputs are of size [64, batch_size, hi(t) of size hidden_dim]

        #inputs  are of size [64, batch_size*hidden_dim]
        tag_space = self.hidden2tag(lstm_out.view(64, -1))
        tag_scores = F.log_softmax(tag_space, dim=1)

        return tag_scores

In [ ]:
model = LSTMratings(embedding_dim, HIDDEN_DIM, 11)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [ ]:
for epoch in range(300):  
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()